<a href="https://colab.research.google.com/github/giterinhub/vectorsearch-applications/blob/main/All_in_one_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine Tune a Sentence Transformer Embedding Model

In [2]:
!git clone https://github.com/giterinhub/vectorsearch-applications.git

Cloning into 'vectorsearch-applications'...
remote: Enumerating objects: 2077, done.
remote: Counting objects: 100% (420/420), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 2077 (delta 366), reused 340 (delta 309), pack-reused 1657
Receiving objects: 100% (2077/2077), 379.45 MiB | 19.44 MiB/s, done.
Resolving deltas: 100% (1268/1268), done.
Updating files: 100% (82/82), done.


In [3]:
!mv vectorsearch-applications/* vectorsearch-applications/.* /content/

mv: cannot move 'vectorsearch-applications/.' to '/content/.': Device or resource busy
mv: cannot move 'vectorsearch-applications/..' to '/content/..': Device or resource busy


In [4]:
!pip install -r /content/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.6/877.6 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.7/798.7 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.1/375.1 kB 42.9 MB/s eta 0:00:00
     ━━━━━

In [ ]:
# !unzip /content/notebooks/model.zip -d /content

In [ ]:
import sys
sys.path.append('../')

from typing import Any
import time
import os

# utilities
from tqdm import tqdm
from rich import print

Fine Tuning

In [ ]:

# import sys
# sys.path.append('../')
try:
  from src.preprocessor.preprocessing import FileIO
except ModuleNotFoundError:
  from preprocessing import FileIO

from torch import cuda
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, losses, InputExample, models

In [ ]:
# Execute Model Loading func
# my_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
#my_model_name = 'Salesforce/SFR-Embedding-Mistral'
my_model_name='Alibaba-NLP/gte-large-en-v1.5'
def load_pretrained_model(model_name: str=my_model_name):
    '''
    Loads sentence transformer modules and returns a pretrained
    model for finetuning.
    '''
    word_embedding_model = models.Transformer(model_name)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model

In [ ]:
model = load_pretrained_model()
model.device

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for Alibaba-NLP/gte-large-en-v1.5 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Alibaba-NLP/gte-large-en-v1.5.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
The repository for Alibaba-NLP/gte-large-en-v1.5 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Alibaba-NLP/gte-large-en-v1.5.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


device(type='cpu')

In [ ]:
model.device

device(type='cpu')

In [ ]:
# Import Training Dataset
#depending on if you are running locally or on Colab
data_path  = 'data/qa_training_triplets.json'  # '/content/qa_training_triplets.json'
data = FileIO.load_json(data_path)
len(data)

500

In [ ]:
# Peek at the data
data[0]

{'positive': 'What effects does L-Cetyl-L-Carnitine have on cellular metabolism?',
 'hard_negative': 'What effects does L-Cetyl-L-Carnitine have on hair growth?',
 'anchor': "I confess I have used it in pill form from time to time, but in part because of the fat oxidation effects, but also because of the other effects that it tends to have. So in exploring the effects that L-Cetyl-L-Carnitine has, it has a huge variety of effects on cellular metabolism. It can reduce ammonia in the blood. That is actually a quite strong effect. It can reduce things like C-reactive protein, which is you want C-reactive protein levels to be managed. You do not want them too high. It can slightly reduce blood glucose. It can slightly increase HDLC, the good form of the blood lipid, and slightly reduce overall cholesterol. And as I mentioned, it can slightly modify the pathway involving glucagon such that you get a considerable effect, not a huge effect on fat oxidation, so it can improve fat oxidation rat

In [ ]:
# Build list of InputExamples & Create Dataloader
train_examples = [InputExample(texts=[sample['anchor'],
                                      sample['positive'],
                                      sample['hard_negative']
                                     ]) for sample in data]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32, )

In [ ]:
# Training example peek
train_examples[0].__dict__

{'guid': '',
 'texts': ["I confess I have used it in pill form from time to time, but in part because of the fat oxidation effects, but also because of the other effects that it tends to have. So in exploring the effects that L-Cetyl-L-Carnitine has, it has a huge variety of effects on cellular metabolism. It can reduce ammonia in the blood. That is actually a quite strong effect. It can reduce things like C-reactive protein, which is you want C-reactive protein levels to be managed. You do not want them too high. It can slightly reduce blood glucose. It can slightly increase HDLC, the good form of the blood lipid, and slightly reduce overall cholesterol. And as I mentioned, it can slightly modify the pathway involving glucagon such that you get a considerable effect, not a huge effect on fat oxidation, so it can improve fat oxidation rates. It has a number of other effects, some of which I talked about during the month on hormones and that sort of thing. It has strong effects on rates

In [ ]:
# Set Loss Function, Epochs, and warm-up
num_epochs = 3
train_loss = losses.MultipleNegativesRankingLoss(model=model)
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [ ]:
# Train model
model.device

device(type='cpu')

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import gc
del variables
gc.collect()

NameError: name 'variables' is not defined

In [ ]:
tf.executing_eagerly()

NameError: name 'tf' is not defined

In [ ]:
os.kill(os.getpid(), 9)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
# Save the model
# Save the finetuned model in current dir
#define your path
model_path = './sfrsem'
model.save(model_path, model_name='gte-large-en-v1.5-ft')
# model.save(path='models', model_name='allminilm-finetuned-256')


In [ ]:
# Zip the model folder into a single file
# ensure the paths match
# !zip -r /content/model.zip allminilm-finetuned-256/
!zip -r /content/sfrsem.zip sfrsem/
# !zip -r model.zip models

  adding: sfrsem/ (stored 0%)
  adding: sfrsem/tokenizer.json (deflated 71%)
  adding: sfrsem/tokenizer_config.json (deflated 74%)
  adding: sfrsem/config_sentence_transformers.json (deflated 27%)
  adding: sfrsem/vocab.txt (deflated 53%)
  adding: sfrsem/1_Pooling/ (stored 0%)
  adding: sfrsem/1_Pooling/config.json (deflated 47%)
  adding: sfrsem/special_tokens_map.json (deflated 80%)
  adding: sfrsem/sentence_bert_config.json (deflated 4%)
  adding: sfrsem/modules.json (deflated 53%)
  adding: sfrsem/config.json (deflated 62%)
  adding: sfrsem/model.safetensors (deflated 8%)
  adding: sfrsem/README.md (deflated 58%)


Now create a Parquet File using the finetuned model

In [ ]:
%load_ext autoreload
%autoreload 2

#standard libraries
import sys
sys.path.append('../')

import os
import time
from typing import List, Tuple
from math import ceil

#external libraries
import pandas as pd
import numpy as np
from rich import print
from rich.pretty import pprint #nifty library for pretty printing
from torch import cuda
from tqdm import tqdm

#external files
try:
    from preprocessing import FileIO
except ModuleNotFoundError:
    from src.preprocessor.preprocessing import FileIO

Import YouTube/Podcast Transcripts

In [ ]:
#root folder on Google Colab is: /content/
root_folder = 'data'  #'../data' / /content/
data_file = 'huberman_labs.json'
data_path = os.path.join(root_folder, data_file)
data_path

'data/huberman_labs.json'

In [ ]:
data = FileIO.load_json(data_path)
print(f'Total # of episodes: {len(data)}')

Total # of episodes: 193

Chunking

In [ ]:
from llama_index.text_splitter import SentenceSplitter #one of the best on the market
import tiktoken # bad ass tokenizer library for use with OpenAI LLMs

#instantiate tokenizer for use with ChatGPT-3.5-Turbo
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo-0125')

#set chunk size and instantiate your SentenceSplitter
chunk_size = 256
gpt35_txt_splitter = SentenceSplitter(chunk_size=chunk_size, tokenizer=encoding.encode, chunk_overlap=0)

In [ ]:
def split_contents(corpus: list[dict],
                   text_splitter: SentenceSplitter,
                   content_field: str='content'
                   ) -> list[list[str]]:
    '''
    Given a corpus of "documents" with text content, this function splits the
    content field into chunks sizes as specified by the text_splitter.

    Example
    -------
    corpus = [
            {'title': 'This is a cool show', 'content': 'There is so much good content on this show. \
              This would normally be a really long block of content. ... But for this example it will not be.'},
            {'title': 'Another Great Show', 'content': 'The content here is really good as well.  If you are \
              reading this you have too much time on your hands. ... More content, blah, blah.'}
           ]

    output = split_contents(data, text_splitter, content_field="content")

    output >>> [['There is so much good content on this show.', 'This would normally be a really long block of content.', \
                 'But for this example it will not be'],
                ['The content here is really good as well.', 'If you are reading this you have too much time on your hands.', \
                 'More content, blah, blah.']
                ]
    '''

    ########################
    # START YOUR CODE HERE #
    ########################
    return [text_splitter.split_text(d[content_field]) for d in tqdm(corpus, 'CHUNKING')]

In [ ]:
content_splits = split_contents(data, gpt35_txt_splitter)

CHUNKING: 100%|██████████| 193/193 [00:19<00:00,  9.65it/s]


Optional Unit testing

In [ ]:
import unittest
from unitesting_utils import load_podcast_data

class TestSplitContents(unittest.TestCase):
    '''
    Unit test to ensure proper functionality of split_contents function
    '''

    def test_split_contents(self):
        import tiktoken
        from llama_index.text_splitter import SentenceSplitter

        data = load_podcast_data(dataset_name='huberman_labs.json')

        subset = data[:3]
        chunk_size = 256
        chunk_overlap = 0
        encoding = tiktoken.encoding_for_model('gpt-3.5-turbo-0125')
        gpt35_txt_splitter = SentenceSplitter(chunk_size=chunk_size, tokenizer=encoding.encode, chunk_overlap=chunk_overlap)
        results = split_contents(subset, gpt35_txt_splitter)
        self.assertEqual(len(results), 3)
        self.assertEqual(len(results[0]), 160)
        self.assertEqual(len(results[1]), 87)
        self.assertEqual(len(results[2]), 152)
        self.assertTrue(isinstance(results, list))
        self.assertTrue(isinstance(results[0], list))
        self.assertTrue(isinstance(results[0][0], str))
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestSplitContents))

Now encode with Finetuned Model

In [ ]:
from sentence_transformers import SentenceTransformer

#define the model you want to use
# model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model_name = model_path # ./sfrsem
model = SentenceTransformer(model_name)
model

The repository for ./sfrsem/ contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/./sfrsem/.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The repository for ./sfrsem/ contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/./sfrsem/.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: NewModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
str_device='cuda:0' if cuda.is_available() else 'cpu'

def encode_content_splits(content_splits: list[list[str]],
                          model: SentenceTransformer,
                          device: str=str_device
                          ) -> list[list[tuple[str, list[float]]]]:
    '''
    Encode content splits as vector embeddings from a vectors of content splits
    where each vectors of splits is a single podcast episode.

    Example
    -------
    content_splits =  [['There is so much good content on this show.', 'This would normally be a really long block of content.'],
                       ['The content here is really good as well.', 'More content, blah, blah.']
                      ]

    output = encode_content_splits(content_splits, model)

    output >>> [
          EPISODE 1 -> [('There is so much good content on this show.',[ 1.78036056e-02, -1.93265956e-02,  3.61164124e-03, -5.89650944e-02,
                                                                         1.91510320e-02,  1.60808843e-02,  1.13610983e-01,  3.59948091e-02,
                                                                        -1.73066761e-02, -3.30348089e-02, -1.00898169e-01,  2.34847311e-02]
                                                                        )
                         tuple(text, vectors), tuple(text, vectors), tuple(text, vectors)....],
          EPISODE 2 ->  [tuple(text, vectors), tuple(text, vectors), tuple(text, vectors)....],
          EPISODE n ... [tuple(text, vectors), tuple(text, vectors), tuple(text, vectors)....]
    '''

    text_vector_tuples = []

    ########################
    # START YOUR CODE HERE #
    ########################
    for episode in tqdm(content_splits, 'VECTORS'):
      vectors = model.encode(episode).tolist()
      text_vector_tuples.append(list(zip(episode, vectors)))
    return text_vector_tuples

In [ ]:
# mount it
from google.colab import drive
drive.mount('/content/drive')
# copy it there
!cp sfrsem.zip /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# model = SentenceTransformer('/workspaces/vectorsearch-applications/notebooks/models/')
# model.device.type='cpu'
text_vector_tuples = encode_content_splits(content_splits, model, device=str_device)

VECTORS: 100%|██████████| 193/193 [05:50<00:00,  1.82s/it]


unit test

In [ ]:
import unittest
from unitesting_utils import load_podcast_data

class TestEncodeContentSplits(unittest.TestCase):
    '''
    Unit test to ensure proper functionality of split_contents function
    '''

    def test_encode_content_splits(self):
        import tiktoken
        from numpy import ndarray
        from llama_index.text_splitter import SentenceSplitter
        from sentence_transformers import SentenceTransformer
# data_path
        data = load_podcast_data(dataset_name='huberman_labs.json')

        #get splits first
        subset = data[:3]
        chunk_size = 256
        chunk_overlap = 0
        encoding = tiktoken.encoding_for_model('gpt-3.5-turbo-0125')
        gpt35_txt_splitter = SentenceSplitter(chunk_size=chunk_size, tokenizer=encoding.encode, chunk_overlap=chunk_overlap)
        splits = split_contents(subset, gpt35_txt_splitter)

        #encode splits
#        model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        # model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        results = encode_content_splits(splits, model)

        #run assertion tests
        self.assertEqual(len(results), 3)
        self.assertEqual(len(results[0]), 160)
        self.assertEqual(len(results[1]), 87)
        self.assertEqual(len(results[2]), 152)
        self.assertTrue(isinstance(results, list))
        self.assertTrue(isinstance(results[0], list))
        self.assertTrue(isinstance(results[0][0], tuple))
        self.assertTrue(isinstance(results[0][0][0], str))
        self.assertTrue(isinstance(results[0][0][1], list))
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestEncodeContentSplits))

Data not available at /content/huberman_labs.json
Data not available at /data/huberman_labs.json, downloading from source


VECTORS: 100%|██████████| 3/3 [00:05<00:00,  1.94s/it]
.
----------------------------------------------------------------------
Ran 1 test in 7.892s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

join metadata

In [ ]:
def join_docs(corpus: List[dict],
              tuples: List[Tuple[str, float]],
              unique_id_field: str='video_id',
              content_field: str='content',
              embedding_field: str='content_embedding'
              ) -> List[dict]:
    docs = []
    for i, d in enumerate(corpus):
        for j, episode in enumerate(tuples[i]):
            doc = {k: v for k, v in d.items() if k != content_field}
            unique_id = doc[unique_id_field]
            doc['doc_id'] = f'{unique_id}_{j}'
            doc[content_field] = episode[0]
            embedding = episode[1]
            assert isinstance(embedding, list), f"Embeddings must be of type list, found type {type(embedding)} instead"
            doc[embedding_field] = embedding
            docs.append(doc)
    return docs

In [ ]:
def join_metadata(corpus: list[dict],
                  text_vector_list: list[list[tuple[str, list]]],
                  unique_id_field: str='video_id',
                  content_field: str='content',
                  embedding_field: str='content_embedding'
                 ) -> list[dict]:
    '''
    Combine episode metadata from original corpus with text/vectors tuples.
    Creates a new dictionary for each text/vector combination.
    '''

    joined_documents = []

    ########################
    # START YOUR CODE HERE #
    ########################

    joined_documents = join_docs(corpus, text_vector_list)

    return joined_documents

In [ ]:
#create your dataset when ready
docs = join_metadata(data, text_vector_tuples)
len(docs)

23905

more unit test

In [ ]:
import unittest
from unitesting_utils import load_podcast_data

class TestJoinMetaData(unittest.TestCase):
    '''
    Unit test to ensure proper functionality of split_contents function
    '''

    def test_join_metadata(self):
        import tiktoken
        from numpy import ndarray
        from llama_index.text_splitter import SentenceSplitter
        from sentence_transformers import SentenceTransformer
# data_path
        data = load_podcast_data(dataset_name='huberman_labs.json')

        #get splits first
        subset = data[:3]
        chunk_size = 256
        chunk_overlap = 0
        encoding = tiktoken.encoding_for_model('gpt-3.5-turbo-0613')
        gpt35_txt_splitter = SentenceSplitter(chunk_size=chunk_size, tokenizer=encoding.encode, chunk_overlap=chunk_overlap)
        splits = split_contents(subset, gpt35_txt_splitter)

        #encode splits
        # model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        text_vector_tuples = encode_content_splits(splits, model)

        #joint metadata
        results = join_metadata(subset, text_vector_tuples)
        #run assertion tests
        self.assertEqual(len(results), 399)
        self.assertEqual(len(results[0]), 13)
        self.assertTrue(isinstance(results, list))
        self.assertTrue(isinstance(results[0], dict))
        self.assertTrue(isinstance(results[0]['content'], str))
        self.assertEqual(results[300]['doc_id'], 'kpTJqwIfHcM_53')
        self.assertEqual(len(results[0]['content_embedding']), 384)
        self.assertTrue(isinstance(results[0]['content_embedding'], list))
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestJoinMetaData))

Data not available at /content/huberman_labs.json
Data not available at /data/huberman_labs.json, downloading from source


VECTORS: 100%|██████████| 3/3 [00:05<00:00,  1.94s/it]
F
FAIL: test_join_metadata (__main__.TestJoinMetaData)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_5507/3972818829.py", line 38, in test_join_metadata
    self.assertEqual(len(results[0]['content_embedding']), 384)
AssertionError: 1024 != 384

----------------------------------------------------------------------
Ran 1 test in 6.320s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

dataset pipeline

In [ ]:
def create_dataset(corpus: list[dict],
                   embedding_model: SentenceTransformer,
                   text_splitter: SentenceSplitter,
                   save_to_disk: bool,
                   file_outpath: str=None,
                   unique_id_field: str='video_id',
                   content_field: str='content',
                   embedding_field: str='content_embedding',
                   device: str=str_device
                   ) -> list[dict]:

    '''
    Given a raw corpus of data, this function creates a new dataset where each dataset
    doc contains episode metadata and it's associated text chunk and vector representation.
    Output is directly saved to disk.
    '''
    if save_to_disk and not file_outpath:
        raise ValueError(f'Saving to disk is enabled but file_outpath was left as a None value.\n\
            Enter a valid file_outpath or mark save_to_disk as False')

    io = FileIO()

    chunk_size = text_splitter.chunk_size
    print(f'Creating dataset using chunk_size: {chunk_size}')
    start = time.perf_counter()
    ########################
    # START YOUR CODE HERE #
    ########################
    content_splits = split_contents(corpus, text_splitter, content_field)
    text_vector_tuples = encode_content_splits(content_splits, embedding_model, device)
    joined_docs = join_docs(corpus, text_vector_tuples, unique_id_field, content_field, embedding_field)

    ########################
    # END YOUR CODE HERE #
    ########################
    if save_to_disk:
        io.save_as_parquet(file_path=file_outpath, data=joined_docs, overwrite=False)
    end = time.perf_counter() - start
    print(f'Total Time to process dataset of chunk_size ({chunk_size}): {round(end/60, 2)} minutes')
    return joined_docs

In [ ]:
dataset = create_dataset(data, model, gpt35_txt_splitter, True, '../huberman_subset_minilm_256.parquet')

Creating dataset using chunk_size: 256

VECTORS: 100%|██████████| 193/193 [05:49<00:00,  1.81s/it]
2024-05-27 14:02:00.655 | INFO     | src.preprocessor.preprocessing:save_as_parquet:42 - DataFrame saved as parquet file here: ../huberman_subset_minilm_256.parquet


Total Time to process dataset of chunk_size (256): 6.23 minutes

In [ ]:
import unittest
from unitesting_utils import load_podcast_data

class TestCreateDataset(unittest.TestCase):
    '''
    Unit test to ensure proper functionality of split_contents function
    '''

    def test_create_dataset(self):
        import tiktoken
        from numpy import ndarray
        from llama_index.text_splitter import SentenceSplitter
        from sentence_transformers import SentenceTransformer

        #data ingest
        data = load_podcast_data(dataset_name='huberman_labs.json')
        subset = data[:3]

        #set constants
        chunk_size = 256
        chunk_overlap = 0
        #encoder, splitter, embedding model
        encoding = tiktoken.encoding_for_model('gpt-3.5-turbo-0125')
        gpt35_txt_splitter = SentenceSplitter(chunk_size=chunk_size, tokenizer=encoding.encode, chunk_overlap=chunk_overlap)
        #model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

        #execute test
        dataset = create_dataset(subset, model, gpt35_txt_splitter, save_to_disk=False)
        #run assertion tests
        self.assertEqual(len(dataset), 399)
        self.assertEqual(len(dataset[0]), 13)
        self.assertTrue(isinstance(dataset, list))
        self.assertTrue(isinstance(dataset[0], dict))
        self.assertTrue(isinstance(dataset[0]['content'], str))
        self.assertEqual(dataset[300]['doc_id'], 'kpTJqwIfHcM_53')
        self.assertEqual(len(dataset[0]['content_embedding']), 384)
        self.assertTrue(isinstance(dataset[0]['content_embedding'], list))
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestCreateDataset))

Data not available at /content/huberman_labs.json
Data not available at /data/huberman_labs.json, downloading from source


Creating dataset using chunk_size: 256

VECTORS: 100%|██████████| 3/3 [00:05<00:00,  1.93s/it]


Total Time to process dataset of chunk_size (256): 0.1 minutes

F
FAIL: test_create_dataset (__main__.TestCreateDataset)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_5507/2911422742.py", line 36, in test_create_dataset
    self.assertEqual(len(dataset[0]['content_embedding']), 384)
AssertionError: 1024 != 384

----------------------------------------------------------------------
Ran 1 test in 6.574s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

save the hard work

In [ ]:
#instantiate FileIO Class
io = FileIO()

#Define your output path
outpath = 'my_ft.parquet'

#save to disk
io.save_as_parquet(file_path=outpath, data=docs, overwrite=False)

2024-05-27 14:02:32.931 | INFO     | src.preprocessor.preprocessing:save_as_parquet:42 - DataFrame saved as parquet file here: my_ft.parquet


In [ ]:
#!mv huberman_subset_minilm_256.parquet new.parquet
!cp my_ft.parquet /content/drive/MyDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Verify that you can reload data

data_with_vectors = io.load_parquet(file_path=outpath)
data_with_vectors[0]

Shape of data: (23905, 13)
Memory Usage: 2.37+ MB


{'video_id': '-OBCwiPPfEU',
 'title': 'Dr. Matthew Walker: The Biology of Sleep & Your Unique Sleep Needs | Huberman Lab Guest Series',
 'length_seconds': 10773,
 'keywords': ['andrew huberman',
  'huberman lab podcast',
  'huberman podcast',
  'dr. andrew huberman',
  'neuroscience',
  'huberman lab',
  'andrew huberman podcast',
  'the huberman lab podcast',
  'science podcast'],
 'view_count': 162689,
 'episode_num': 193,
 'episode_url': 'https://www.youtube.com/watch?v=-OBCwiPPfEU',
 'thumbnail_url': 'https://i.ytimg.com/vi/-OBCwiPPfEU/mqdefault.jpg',
 'guest': 'Dr. Matthew Walker',
 'summary': "In the Huberman Lab podcast episode featuring Dr. Matthew Walker, the discussion centers on the biology of sleep and how it impacts mental and physical health. They delve into the different stages of sleep, including non-REM and REM sleep, and their unique roles in brain function, such as memory consolidation and emotional regulation. They also explore practical strategies for improving sle

## Model Evaluation
Fine-tuning is just the start! You still have to create a new dataset using the fine-tuned model, index that data on Weaviate, and then evaluate its performance. This is why having a solid dataset creation and indexing pipeline is key, especially if you plan on running multiple experiments to optimize your results. Follow this recipe:

1. Create new dataset (from Notebook 1)
1. Index that dataset and create an easily identifiable collection name i.e. Huberman_minilm_finetuned_256 (from Notebook 2)
1. Run the execute_evaluation function (from Notebook 4)

Assuming you are in the notebooks folder when performing the new evaluation and you have created a models folder in the top-level directory, the following code snippet will load the Weaviate client with the fine-tuned model and ensure that you are hitting the right collection for evaluation:

## Create new dataset

In [1]:
# Create new dataset (from Notebook 1)
import sys
sys.path.append('../')

#external files
from src.preprocessor.preprocessing import FileIO
from src.database.weaviate_interface_v4 import WeaviateIndexer, WeaviateWCS
from src.database.database_utils import get_weaviate_client

#standards
import os
import time
import json
from typing import List
from tqdm import tqdm
from rich import print  # nice library that provides improved printing output (overrides default print function)

In [3]:
!cp drive/MyDrive/sfrsem.zip .

In [4]:
!unzip sfrsem.zip

Archive:  sfrsem.zip
   creating: sfrsem/
  inflating: sfrsem/tokenizer.json   
  inflating: sfrsem/tokenizer_config.json  
  inflating: sfrsem/config_sentence_transformers.json  
  inflating: sfrsem/vocab.txt        
   creating: sfrsem/1_Pooling/
  inflating: sfrsem/1_Pooling/config.json  
  inflating: sfrsem/special_tokens_map.json  
  inflating: sfrsem/sentence_bert_config.json  
  inflating: sfrsem/modules.json     
  inflating: sfrsem/config.json      
  inflating: sfrsem/model.safetensors  
  inflating: sfrsem/README.md        


In [5]:
#instantiate client,
#load from local .env file
from google.colab import userdata
api_key = userdata.get('WEAVIATE_API_KEY')
url = userdata.get('WEAVIATE_ENDPOINT')
# userdata.get('OPENAI_API_KEY')

model_path='./sfrsem'

client = WeaviateWCS(endpoint=url, api_key=api_key, model_name_or_path=model_path)
client._client.is_live()


The repository for ./sfrsem/ contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/./sfrsem/.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


The repository for ./sfrsem/ contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/./sfrsem/.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


modeling.py:   0%|          | 0.00/57.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


True

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')
!cp /content/drive/MyDrive/my_ft.parquet .


In [7]:
data_path = 'my_ft.parquet'

data = FileIO.load_parquet(data_path)

Shape of data: (23905, 13)
Memory Usage: 2.37+ MB


In [8]:
from src.database.properties_template import properties

In [9]:
#create your own Collection name or use the example from above
collection_name = 'my_ft3'

In [10]:
client.create_collection(collection_name=collection_name, properties=properties, description='Huberman Labs: 193 full-length transcripts')
# print(client.show_collection_config(collection_name))

Collection "my_ft3" created


Index your data

In [11]:
indexer = WeaviateIndexer(client)
indexer.batch_index_data(data=data, collection_name=collection_name, verbose=True)

100%|██████████| 23905/23905 [01:12<00:00, 330.57it/s] 


Processing finished in 1.68 minutes.
Batch job completed with zero errors.


{'num_errors': 0, 'error_messages': [], 'doc_ids': []}

In [12]:
# Check total doc count
total_docs = client.get_doc_count(collection_name)
# Found 23905 documents in collection "Huberman_minilm_256"

Found 23905 documents in collection "my_ft3"


## Run Evaluation

In [14]:
from src.reranker import ReRanker

In [16]:
reranker = ReRanker(model_name='cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [19]:
# import sys
# sys.path.append('../')
# from src.evaluation.retrieval_evaluation import execute_evaluation
# from src.preprocessor.preprocessing import FileIO
# from retrieval_evaluation import execute_evaluation
# from preprocessing import FileIO
# from src.reranker import ReRanker
from src.evaluation.retrieval_evaluation import execute_evaluation
from src.preprocessor.preprocessing import FileIO

#################
##  START CODE ##
#################

# collection_name='Huberman_minilm_256_ft2'

### Load QA dataset
data_path = 'data/golden_datasets/golden_256_hard.json'
golden_dataset = FileIO.load_json(data_path)

from google.colab import userdata
api_key = userdata.get('WEAVIATE_API_KEY')
# url = os.environ['WEAVIATE_ENDPOINT']
url = userdata.get('WEAVIATE_ENDPOINT')
# model_path = 'sentence-transformers/all-MiniLM-L6-v2'

# model_path = '/content/models'
model_path = 'sfrsem'
client = WeaviateWCS(endpoint=url, api_key=api_key, model_name_or_path=model_path)
client._client.is_live()

# reranker = ReRanker(model_name='cross-encoder/ms-marco-MiniLM-L-6-v2')
reranker = ReRanker(model_name='BAAI/bge-reranker-base')

retrieval_results = execute_evaluation(
    dataset=golden_dataset,
    collection_name=collection_name,
    retriever=client,
    chunk_size=256,
    alpha=0.54
    )

#################
##  END CODE   ##
#################
print(retrieval_results)

The repository for sfrsem/ contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/sfrsem/.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
The repository for sfrsem/ contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/sfrsem/.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Queries: 100%|██████████| 100/100 [01:17<00:00,  1.28it/s]


Total Processing Time: 1.3 minutes

{
    'n': 100,
    'top_k': 5,
    'alpha': 0.54,
    'Retriever': 'sfrsem',
    'Ranker': 'None',
    'chunk_size': 256,
    'query_props': ['content'],
    'total_misses': 9,
    'total_questions': 100,
    'kw_hit_rate': 0.78,
    'kw_mrr': 0.7,
    'vector_hit_rate': 0.82,
    'vector_mrr': 0.69,
    'hybrid_hit_rate': 0.87,
    'hybrid_mrr': 0.77
}